In [1]:
import pandas as pd
import warnings
from pandas import Series, DataFrame
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [3]:

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBClassifier
from time import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [31]:
def filter1(s,f):
    if s==f:
        return 0
    else:
        return float(s)
def transfrom(input_data):
    input_data['Coupon_requirement']=input_data['coupon_requirement'].map(lambda x: filter1(x,'null'))
    input_data['Equivalent_discount']=input_data['equivalent_discount'].map(lambda x: filter1(x,'fixed'))
    input_data['Fixed']=input_data['equivalent_discount'].map(lambda x: int(x=='fixed'))
    return input_data.drop(['coupon_requirement','equivalent_discount'],axis=1)

In [39]:

def fillna_pipeline(input_data):
    input_data=transfrom(input_data)
    input_data['Num_coupon_used'].fillna(0,inplace=True)
    input_data['Difference'].fillna(input_data['Difference'].mean(),inplace=True)
    input_data['Tot_coupon_sent'].fillna(input_data['Tot_coupon_sent'].mean(),inplace=True)
    input_data['num_type'].fillna(input_data['num_type'].mean(),inplace=True)
    input_data["Customer's Total Usage for Merchant's Coupons"].fillna(input_data["Customer's Total Usage for Merchant's Coupons"].mean(),inplace=True)
    input_data["Customer's Total Usage Fraction for Merchant"].fillna(input_data["Customer's Total Usage Fraction for Merchant"].mean(),inplace=True)
    input_data["Customer's Total Received by Merchant"].fillna(input_data["Customer's Total Received by Merchant"].mean(),inplace=True)
    input_data["Merchant's Total Usage Fraction"].fillna(input_data["Merchant's Total Usage Fraction"].mean(),inplace=True)
    input_data["Customer's Inbox Share by Merchant"].fillna(input_data["Customer's Inbox Share by Merchant"].mean(),inplace=True)
    print 'fill completed'
    return input_data

In [26]:
y_train1=pd.read_csv('y_train1.csv',header=None)
y_test1=pd.read_csv('y_test1.csv',header=None)

In [40]:
X_train1=fillna_pipeline(pd.read_csv('X_train1.csv'))
X_test1=fillna_pipeline(pd.read_csv('X_test1.csv'))

fill completed
fill completed


In [43]:
GBDT = GradientBoostingClassifier(random_state=42)
model=GBDT.fit(X_train1, y_train1)



In [45]:
y1_GBDTpredicted_test = GBDT.predict(X_test1)
metrics.accuracy_score(y_test1,y1_GBDTpredicted_test)

0.80061310743891601

In [47]:
confusion_matrix(y_test1,y1_GBDTpredicted_test)

array([[114221,  10409],
       [ 44551, 106464]])

In [48]:
106464.0/(44551+106464)

0.7049895705724597

In [ ]:
parameter_candidates = {
    'loss': ('deviance', 'exponential'),
}
GBDT = GradientBoostingClassifier(random_state=42)
GBDTGS = GridSearchCV(
    estimator=GBDT, param_grid=parameter_candidates, cv=3, n_jobs=4)

GBDTGS.fit(X_train1, y_train1)
print GBDTGS.best_params_
print GBDTGS.best_score_